In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('data_encoded.csv')
df

,Company,TypeName,OS,Display_type,CPU_Brand,Primary_type,Secondary_type,GPU,log_price,Weight,Inches,megapixel,RAM,Primary_Storage,Secondary_Storage
0,12,150,12,271,1060,748,1116,646,11.175755,1.37,13.3,4.1,8,128,0
1,12,150,12,763,1060,748,1116,646,10.776777,1.34,13.3,1.3,8,128,0
2,247,704,87,763,1060,748,1116,646,10.329931,1.86,15.6,2.1,8,256,0
3,12,150,12,271,1060,748,1116,646,11.473101,1.37,13.3,4.1,8,256,0
4,92,704,958,763,58,370,1116,171,9.967026,2.10,15.6,1.0,4,500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,260,104,958,271,1060,748,1116,646,10.433899,1.80,14.0,2.1,4,128,0
1114,260,104,958,271,1060,748,1116,646,11.288115,1.30,13.3,5.8,16,512,0
1115,260,704,958,763,1060,748,1116,646,9.409283,1.50,14.0,1.0,2,64,0
1116,247,704,958,763,1060,370,1116,171,10.614129,2.19,15.6,1.0,6,1024,0


In [5]:
X = df.drop(columns='log_price')
y = df['log_price']

In [7]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
from sklearn.linear_model import LinearRegression

In [13]:
lr = LinearRegression()

In [15]:
lr.fit(X_train,y_train)

LinearRegression()

In [17]:
lr.score(X_test,y_test)

0.5889358695807776

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

In [21]:
models = {
    'LinearRegression' : LinearRegression(),
    'Random Forest' : RandomForestRegressor(),
    'Ada Boost' : AdaBoostRegressor(),
    'Decision Tree' : DecisionTreeRegressor(),
    'XGBRegressor' : xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
    'GradientBoostingRegressor' : GradientBoostingRegressor(random_state=42),
    'KNeighborsRegressor' : KNeighborsRegressor(n_neighbors=5),
    'MLPRegressor': MLPRegressor(hidden_layer_sizes=(100, 100), activation='relu', max_iter=500)
}

In [23]:
from sklearn import metrics

def print_evaluation(actual,pred):
    mae = metrics.mean_absolute_error(actual,pred)
    mse = metrics.mean_squared_error(actual,pred)
    rmse = np.sqrt(metrics.mean_squared_error(actual,pred))
    r2 = metrics.r2_score(actual,pred)*100
    
    print('MAE :',mae)
    print('MSE :',mse)
    print('RMSE :',rmse)
    print('R2 :',r2)
    
    return r2

In [25]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    print(list(models.values())[i])
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print_evaluation(y_test,y_pred)
    print('\n')

LinearRegression()
MAE : 0.2826748199430528
MSE : 0.1291724262298951
RMSE : 0.35940565692528426
R2 : 58.89358695807776


RandomForestRegressor()
MAE : 0.18637206377823037
MSE : 0.05946300657611977
RMSE : 0.2438503774369024
R2 : 81.07714641294848


AdaBoostRegressor()
MAE : 0.2470650747295243
MSE : 0.09314724391251794
RMSE : 0.3052003340635753
R2 : 70.35784498488825


DecisionTreeRegressor()
MAE : 0.21676387065513428
MSE : 0.07821885477074018
RMSE : 0.2796763393116053
R2 : 75.10849145041419


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_ste

### Selecting xgbregressor, as it has highest R2 score. HyperParameter tuning

In [27]:
from sklearn.model_selection import GridSearchCV

In [29]:
rf = RandomForestRegressor(random_state=42)

In [31]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [33]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [35]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


C:\Users\yashw\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
324 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
193 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\yashw\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\yashw\anaconda3\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "C:\Users\yashw\anaconda3\Lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\yashw\anaconda3\Lib\site-packag

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error', verbose=2)

In [39]:
print("Best Hyperparameters:", grid_search.best_params_)

# Predict with the best estimator
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Evaluate the model
mse = np.mean((y_test - y_pred)**2)
print("Mean Squared Error:", mse)

Best Hyperparameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Mean Squared Error: 0.05793219989283237


In [41]:
best_rf.score(X_test,y_test)

0.8156429350499548

# Gradientboost

In [44]:
gb = GradientBoostingRegressor(random_state=42)

In [46]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0],
    'loss': ['squared_error', 'absolute_error']
}

In [48]:
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [50]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 972 candidates, totalling 2916 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'loss': ['squared_error', 'absolute_error'],
                         'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=2)

In [52]:
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300, 'subsample': 0.8}


In [56]:
best_gb = grid_search.best_estimator_
best_gb

GradientBoostingRegressor(min_samples_leaf=4, min_samples_split=10,
                          n_estimators=300, random_state=42, subsample=0.8)

In [58]:
y_pred = best_gb.predict(X_test)

In [60]:
mse = np.mean((y_test - y_pred)**2)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.05435323422223809


In [62]:
best_gb.score(X_test,y_test)

0.8270322419951129

In [64]:
import pickle

In [66]:
with open('model_v1.pkl', 'wb') as file:
    pickle.dump(best_gb, file)

In [68]:
df.describe()

,Company,TypeName,OS,Display_type,CPU_Brand,Primary_type,Secondary_type,GPU,log_price,Weight,Inches,megapixel,RAM,Primary_Storage,Secondary_Storage
count,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000
mean,205.062612,487.250447,831.125224,592.724508,1008.017889,622.901610,1114.007156,480.463327,10.716347,1.967030,14.988551,2.049732,7.527728,454.926655,138.740608
std,86.787919,283.631890,310.694530,253.730948,222.324417,177.949172,47.096084,197.958534,0.562925,0.440855,1.257904,1.251204,3.636904,377.288322,382.776123
min,2.000000,18.000000,12.000000,84.000000,58.000000,370.000000,2.000000,171.000000,9.134616,1.180000,11.600000,1.000000,2.000000,8.000000,0.000000
25%,120.000000,150.000000,958.000000,271.000000,1060.000000,370.000000,1116.000000,301.000000,10.319310,1.600000,14.000000,1.000000,4.000000,256.000000,0.000000
50%,247.000000,704.000000,958.000000,763.000000,1060.000000,748.000000,1116.000000,646.000000,10.765658,2.040000,15.600000,2.100000,8.000000,256.000000,0.000000
75%,260.000000,704.000000,958.000000,763.000000,1060.000000,748.000000,1116.000000,646.000000,11.143946,2.230000,15.600000,2.100000,8.000000,512.000000,0.000000
max,272.000000,704.000000,958.000000,763.000000,1060.000000,748.000000,1116.000000,646.000000,11.777361,2.900000,17.300000,8.300000,24.000000,2048.000000,2048.000000


In [70]:
df.columns

Index(['Company', 'TypeName', 'OS', 'Display_type', 'CPU_Brand',
       'Primary_type', 'Secondary_type', 'GPU', 'log_price', 'Weight',
       'Inches', 'megapixel', 'RAM', 'Primary_Storage', 'Secondary_Storage'],
      dtype='object')